In [4]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
import PyPDF2

In [5]:
load_dotenv()
KEY = os.getenv("OPENAI_API_KEY")


In [6]:
llm = ChatOpenAI(openai_api_key= KEY ,model_name='gpt-3.5-turbo',temperature=0.3)

/var/folders/11/0l10cpcs5wz0bsg7gzqwp5k80000gn/T/ipykernel_42961/4196351380.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(openai_api_key= KEY ,model_name='gpt-3.5-turbo',temperature=0.3)


In [7]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
        "explain":"explaination"
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
        "explain":"explaination"
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
        "explain":"explaination"
    },
}

In [8]:
TEMPLATE = """
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
For each question, provide:
- The question
- Four options (a, b, c, d)
- The correct answer (as a, b, c, or d)
- A brief explanation for the correct answer

Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Format your response like the RESPONSE_JSON below and use it as a guide.
Ensure to make {number} MCQs.
### RESPONSE_JSON
{response_json}
"""

In [9]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [10]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

/var/folders/11/0l10cpcs5wz0bsg7gzqwp5k80000gn/T/ipykernel_42961/2669661367.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)


In [11]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [12]:
quiz_evaluation_prompt=PromptTemplate(
    input_variables=["subject", "quiz"],
    template=TEMPLATE
) 

In [13]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True) 

In [14]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [15]:
file_path=r"/Users/b.anasurya/Desktop/Generative AI/MCQ_generator/data.txt"

In [16]:
file_path

'/Users/b.anasurya/Desktop/Generative AI/MCQ_generator/data.txt'

In [31]:
with open(file_path,'r') as file:
    TEXT = file.read() 

In [32]:
print(TEXT)

"Biological" redirects here. For other uses, see Biological (disambiguation) and Biology (disambiguation).






Clockwise from upper left: A Nile crocodile, the common sunflower, Escherichia coli, Amoeba proteus, the fruit fly, and Sarcoscypha coccinea
Part of a series on
Biology

IndexOutlineGlossary
History (timeline)
Key components
Branches
Research
Applications
icon Biology portal Category
vte
Biology is the scientific study of life and living organisms. It is a broad natural science that encompasses a wide range of fields and unifying principles that explain the structure, function, growth, origin, evolution, and distribution of life. Central to biology are five fundamental themes: the cell as the basic unit of life, genes and heredity as the basis of inheritance, evolution as the driver of biological diversity, energy transformation for sustaining life processes, and the maintenance of internal stability (homeostasis).[1][2]

Biology examines life across multiple levels of organ

In [33]:
# Serialize the python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer", "explain": "explaination"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer", "explain": "explaination"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer", "explain": "explaination"}}'

In [34]:
NUMBER=5 
SUBJECT="Biology"
TONE="Hard"

In [35]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
)

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:

Text:"Biological" redirects here. For other uses, see Biological (disambiguation) and Biology (disambiguation).






Clockwise from upper left: A Nile crocodile, the common sunflower, Escherichia coli, Amoeba proteus, the fruit fly, and Sarcoscypha coccinea
Part of a series on
Biology

IndexOutlineGlossary
History (timeline)
Key components
Branches
Research
Applications
icon Biology portal Category
vte
Biology is the scientific study of life and living organisms. It is a broad natural science that encompasses a wide range of fields and unifying principles that explain the structure, function, growth, origin, evolution, and distribution of life. Central to biology are five fundamental themes: the cell as the basic unit of life, genes and heredity as the basis of inheritance, evolution as the driver of biological diversity, energy transformation for sustaining life processes, and the maintenance of internal stability (homeostasis).[1][2]

Biology examines life 

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



> Finished chain.
Prompt after formatting:

Text:"Biological" redirects here. For other uses, see Biological (disambiguation) and Biology (disambiguation).






Clockwise from upper left: A Nile crocodile, the common sunflower, Escherichia coli, Amoeba proteus, the fruit fly, and Sarcoscypha coccinea
Part of a series on
Biology

IndexOutlineGlossary
History (timeline)
Key components
Branches
Research
Applications
icon Biology portal Category
vte
Biology is the scientific study of life and living organisms. It is a broad natural science that encompasses a wide range of fields and unifying principles that explain the structure, function, growth, origin, evolution, and distribution of life. Central to biology are five fundamental themes: the cell as the basic unit of life, genes and heredity as the basis of inheritance, evolution as the driver of biological diversity, energy transformation for sustaining life processes, and the maintenance of internal stability (homeostasis).[1][2]

Bio

In [36]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:8221
Prompt Tokens:7236
Completion Tokens:985
Total Cost:0.012823999999999999


In [37]:
response

{'text': '"Biological" redirects here. For other uses, see Biological (disambiguation) and Biology (disambiguation).\n\n\n\n\n\n\nClockwise from upper left: A Nile crocodile, the common sunflower, Escherichia coli, Amoeba proteus, the fruit fly, and Sarcoscypha coccinea\nPart of a series on\nBiology\n\nIndexOutlineGlossary\nHistory (timeline)\nKey components\nBranches\nResearch\nApplications\nicon Biology portal Category\nvte\nBiology is the scientific study of life and living organisms. It is a broad natural science that encompasses a wide range of fields and unifying principles that explain the structure, function, growth, origin, evolution, and distribution of life. Central to biology are five fundamental themes: the cell as the basic unit of life, genes and heredity as the basis of inheritance, evolution as the driver of biological diversity, energy transformation for sustaining life processes, and the maintenance of internal stability (homeostasis).[1][2]\n\nBiology examines life 

In [38]:
quiz = json.loads(response.get("quiz"))
for q_num, q_data in quiz.items():
    print(f"Q{q_num}: {q_data['mcq']}")
    for opt, text in q_data['options'].items():
        print(f"  {opt}. {text}")
    print(f"Correct: {q_data['correct']}")
    print(f"Explanation: {q_data['explain']}\n")

Q1: What is the fundamental unit of life according to cell theory?
  a. Molecules
  b. Organelles
  c. Cells
  d. Tissues
Correct: c
Explanation: Cell theory states that cells are the fundamental units of life, making option c, 'Cells', the correct answer.

Q2: Which scientist is credited with the theory of evolution by natural selection?
  a. Gregor Mendel
  b. Anton van Leeuwenhoek
  c. Charles Darwin
  d. Louis Pasteur
Correct: c
Explanation: Charles Darwin is credited with the theory of evolution by natural selection, making option c, 'Charles Darwin', the correct answer.

Q3: Which of the following is NOT a macromolecule?
  a. Proteins
  b. Lipids
  c. Sugars
  d. Water
Correct: d
Explanation: Water is not a macromolecule, as it is a small polar molecule, making option d, 'Water', the correct answer.

Q4: What is the function of mitochondria in a cell?
  a. Protein synthesis
  b. Energy production
  c. Storage of genetic material
  d. Cell movement
Correct: b
Explanation: Mitochon

In [39]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    explain = value["explain"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct,"Explanation": explain}) 

In [40]:
quiz_table_data

[{'MCQ': 'What is the fundamental unit of life according to cell theory?',
  'Choices': 'a: Molecules | b: Organelles | c: Cells | d: Tissues',
  'Correct': 'c',
  'Explanation': "Cell theory states that cells are the fundamental units of life, making option c, 'Cells', the correct answer."},
 {'MCQ': 'Which scientist is credited with the theory of evolution by natural selection?',
  'Choices': 'a: Gregor Mendel | b: Anton van Leeuwenhoek | c: Charles Darwin | d: Louis Pasteur',
  'Correct': 'c',
  'Explanation': "Charles Darwin is credited with the theory of evolution by natural selection, making option c, 'Charles Darwin', the correct answer."},
 {'MCQ': 'Which of the following is NOT a macromolecule?',
  'Choices': 'a: Proteins | b: Lipids | c: Sugars | d: Water',
  'Correct': 'd',
  'Explanation': "Water is not a macromolecule, as it is a small polar molecule, making option d, 'Water', the correct answer."},
 {'MCQ': 'What is the function of mitochondria in a cell?',
  'Choices': '

In [41]:
quiz = pd.DataFrame(quiz_table_data)

In [42]:
quiz.to_csv("biology.csv", index=False)

In [43]:
print(type(quiz))

<class 'pandas.core.frame.DataFrame'>


In [45]:
from datetime import datetime

{datetime.now().strftime('%m_%d_%Y_%H_%M_%S')}

{'07_09_2025_09_30_42'}